# OLID datasets

## import libraries

In [1]:
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


## load the datasets

In [2]:
train_data=pd.read_csv('../raw_data/olid-training-v1.0.tsv', delimiter='\t', encoding='utf-8')

train_tweets = train_data[['tweet']] #Extract tweets
train_task_a_labels= train_data[['subtask_a']] #Extract subtsak_a labels
train_task_b_labels= train_data[['subtask_b']] #Extract subtsak_b labels
train_task_c_labels= train_data[['subtask_c']] #Extract subtsak_c labels

train_task_a_labels.columns.values[0] = 'class_a' #Rename class attribute
train_task_b_labels.columns.values[0] = 'class_b' #Rename class attribute
train_task_c_labels.columns.values[0] = 'class_c' #Rename class attribute

print(train_data)

          id                                              tweet subtask_a  \
0      86426  @USER She should ask a few native Americans wh...       OFF   
1      90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2      16820  Amazon is investigating Chinese employees who ...       NOT   
3      62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4      43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   
...      ...                                                ...       ...   
13235  95338  @USER Sometimes I get strong vibes from people...       OFF   
13236  67210  Benidorm ✅  Creamfields ✅  Maga ✅   Not too sh...       NOT   
13237  82921  @USER And why report this garbage.  We don't g...       OFF   
13238  27429                                        @USER Pussy       OFF   
13239  46552  #Spanishrevenge vs. #justice #HumanRights and ...       NOT   

      subtask_b subtask_c  
0           UNT       NaN  
1           TIN    

In [3]:
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [4]:
train_data['subtask_a'].value_counts()

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64

## NLP

In [5]:
!pip install emoji

In [6]:
import emoji
import re

In [7]:
def replace_multiple_user_mentions(text):
    """
    Replaces multiple '@USER' mentions in a tweet with a single '@USERS'.
    """
    user_mentions = len(re.findall('@USER', text))
    if user_mentions > 1:
        text = re.sub('@USER', '@USERS', text)
    return text


In [8]:

def emoji_to_text(text):
    return emoji.demojize(text, delimiters=("", ""))

In [9]:
def clean_tweets(df):
    punctuations = string.punctuation

    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
    df.loc[:, 'tweet'] = df.tweet.str.lower() #Lowercase

    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

        df['tweet'] = df['tweet'].apply(
        lambda x: x.encode('ascii', 'ignore').decode('ascii')
)

    # Transfer emojis to segmentations
    df.loc[:, 'tweet'] = df['tweet'].apply(emoji.demojize)
    #Remove emojis
    #df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces
    # Replace multiple user mentions
    df.loc[:, 'tweet'] = df['tweet'].apply(replace_multiple_user_mentions)

    # Substitute out-of-vocabulary (OOV) words
    df.loc[:, 'tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+|www\S+', 'http', x))


In [10]:
clean_tweets(train_tweets)

/Users/zhenliu15471/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_53505/364354749.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_53505/364354749.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [11]:
train_task_a_data = train_tweets.join(train_task_a_labels)

#train_task_b_data = train_tweets.join(train_task_b_labels)
#train_task_b_data = train_task_b_data.dropna() #Drop records with missing values

#train_task_c_data = train_tweets.join(train_task_c_labels)
#train_task_c_data = train_task_c_data.dropna() #Drop records with missing values

#Apply quotes to cleaned tweets
train_task_a_data.update(train_task_a_data[['tweet']].applymap('\'{}\''.format))
#train_task_b_data.update(train_task_b_data[['tweet']].applymap('\'{}\''.format))
#train_task_c_data.update(train_task_c_data[['tweet']].applymap('\'{}\''.format))

In [12]:
train_task_a_data.head()

,tweet,class_a
0,' she should ask a few native americans what t...,OFF
1,' go home youre drunk maga trump ',OFF
2,'amazon is investigating chinese employees who...,NOT
3,' someone shouldvetaken this piece of shit to ...,OFF
4,' obama wanted liberals and illegals to move ...,NOT


In [13]:
#Read tweets from test sets
test_tweet_a=pd.read_csv('../raw_data/testset-levela.tsv', delimiter='\t', encoding='utf-8')
#test_tweet_b=pd.read_csv('../raw_data/testset-levelb.tsv', delimiter='\t', encoding='utf-8')
#test_tweet_c=pd.read_csv('../raw_data/testset-levelc.tsv', delimiter='\t', encoding='utf-8')

#Read tweet labels
test_label_a=pd.read_csv('../raw_data/labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])
#test_label_b=pd.read_csv('../raw_data/labels-levelb.csv', encoding='utf-8',
                       #  index_col=False, names=['id', 'class_b'])
#test_label_c=pd.read_csv('../raw_data/labels-levelc.csv', encoding='utf-8',
                       #  index_col=False, names=['id', 'class_c'])

#Merge tweets with labels by id
test_tweet_a = test_tweet_a.merge(test_label_a, on='id')
#test_tweet_b = test_tweet_b.merge(test_label_b, on='id')
#test_tweet_c = test_tweet_c.merge(test_label_c, on='id')

#Drop id column
test_tweet_a = test_tweet_a.drop(columns='id')
#test_tweet_b = test_tweet_b.drop(columns='id')
#test_tweet_c = test_tweet_c.drop(columns='id')

#Clean tweets in test sets
clean_tweets(test_tweet_a)
#clean_tweets(test_tweet_b)
#clean_tweets(test_tweet_c)

#Apply quotes to cleaned tweets
test_tweet_a.update(test_tweet_a[['tweet']].applymap('\'{}\''.format))
#test_tweet_b.update(test_tweet_b[['tweet']].applymap('\'{}\''.format))
#test_tweet_c.update(test_tweet_c[['tweet']].applymap('\'{}\''.format))

test_tweet_a.head()

/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_53505/364354749.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_53505/364354749.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


,tweet,class_a
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF
1,'constitutionday is revered by conservatives h...,NOT
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT
3,'watching boomer getting the news that she is ...,NOT
4,'nopasaran unity demo to oppose the farright i...,OFF


# Task A

In [14]:
# Creating labelEncoder
encoder = LabelEncoder()
# Converting string labels into numbers.
train_task_a_data["class_a_code"] = encoder.fit_transform(train_task_a_data["class_a"])
test_tweet_a["class_a_code"] = encoder.fit_transform(test_tweet_a["class_a"])

In [15]:
#Create tuple pair for class and class code
train_task_a_data['class-tuple'] = train_task_a_data[['class_a', 'class_a_code']].apply(tuple, axis=1)
class_a = train_task_a_data['class-tuple'].unique()
class_a

array([('OFF', 1), ('NOT', 0)], dtype=object)

In [16]:
# Label the Data

train_tweets = train_task_a_data['tweet']
train_labels = train_task_a_data['class_a_code']

X_test = test_tweet_a['tweet']
y_test = test_tweet_a['class_a_code']


print("Shape of train_tweet is {} and shape of train_label is {}".format(train_tweets.shape, train_labels.shape))
print("Shape of test_tweet is {} and shape of test_label is {}".format(X_test.shape, y_test.shape))

Shape of train_tweet is (13240,) and shape of train_label is (13240,)
Shape of test_tweet is (860,) and shape of test_label is (860,)


In [17]:
X_train,X_val,y_train,y_val = train_test_split(train_tweets,train_labels,test_size=0.30,random_state=42)


print("Shape of test_tweets is {} and shape of test_labels is {}".format(X_train.shape, y_train.shape))

Shape of test_tweets is (9268,) and shape of test_labels is (9268,)


In [18]:
X_train

2804               '  truly anything to avoid gun control'
13084    ' its not wrongfully characterizing its the fu...
2211     '  they need to add more gun control laws and ...
11154    'boom  maybe that book deal isnt such a good i...
10620    ' unfirtunately i have seen hispanics justifyi...
                               ...                        
11964    ' isnt there a statute of limitations on sexua...
5191             '  he is probably dying due to dog fumes'
5390                                  '  you are so right'
860           ' sams act is working i swear she is acting'
7270                                     '  chuckle heads'
Name: tweet, Length: 9268, dtype: object

In [19]:
y_train

2804     0
13084    1
2211     0
11154    0
10620    1
        ..
11964    0
5191     0
5390     0
860      0
7270     0
Name: class_a_code, Length: 9268, dtype: int64

In [20]:
test_tweet_a

,tweet,class_a,class_a_code
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF,1
1,'constitutionday is revered by conservatives h...,NOT,0
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT,0
3,'watching boomer getting the news that she is ...,NOT,0
4,'nopasaran unity demo to oppose the farright i...,OFF,1
...,...,...,...
855,'despicabledems lie again about rifles dem dis...,OFF,1
856,'meetthespeakers will present in our event o...,NOT,0
857,' people just unfollowed me for talking about ...,OFF,1
858,'wednesdaywisdom antifa calls the right fascis...,NOT,0


In [21]:
X_test

0      'whoisq wherestheserver dumpnike declasfisa de...
1      'constitutionday is revered by conservatives h...
2      'foxnews nra maga potus trump ndamendment rnc ...
3      'watching boomer getting the news that she is ...
4      'nopasaran unity demo to oppose the farright i...
                             ...                        
855    'despicabledems lie again about rifles dem dis...
856    'meetthespeakers   will present in our event o...
857    ' people just unfollowed me for talking about ...
858    'wednesdaywisdom antifa calls the right fascis...
859             'kavanaugh typical liberals  democrats '
Name: tweet, Length: 860, dtype: object

In [22]:
y_test

0      1
1      0
2      0
3      0
4      1
      ..
855    1
856    0
857    1
858    0
859    0
Name: class_a_code, Length: 860, dtype: int64

In [23]:
random_0_rows = test_tweet_a[test_tweet_a['class_a_code'] == 0].sample(n=100, random_state=42)

# Select random 50 rows with class_a_code equal to 1
random_1_rows = test_tweet_a[test_tweet_a['class_a_code'] == 1].sample(n=50, random_state=42)

# Concatenate the two DataFrames
test_tweet_a_150 = pd.concat([random_0_rows, random_1_rows])

In [24]:
test_tweet_a_150= test_tweet_a_150.sample(frac=1, random_state=42).reset_index(drop=True)

In [25]:
test_tweet_a_150

,tweet,class_a,class_a_code
0,'aurora il kanecty please keep looking for and...,NOT,0
1,' same here i cant hear what she is saying',NOT,0
2,'geoffreyowens niggas dont really understandle...,OFF,1
3,'trump may be many things but he is entertaini...,NOT,0
4,'kamalaharris may run for president in but wi...,NOT,0
...,...,...,...
145,'conservatives you cannot ride this one outvot...,NOT,0
146,' beats the hell nj out of me lol dude alre...,OFF,1
147,'watching boomer getting the news that she is ...,NOT,0
148,'brexit is dead not because the other eu state...,NOT,0


In [26]:
X_test_150 = test_tweet_a_150['tweet']
y_test_150 = test_tweet_a_150['class_a_code']

In [27]:
!pip install transformers



[notice] A new release of pip available: 22.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [28]:
from transformers import BertTokenizerFast, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

# Use the same tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Encode Labels
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)
val_labels = encoder.transform(y_val)
test_labels = encoder.transform(y_test)

# Tokenize and align labels with the tokenized inputs
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=256)



/Users/zhenliu15471/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [206]:
y_test

0      1
1      0
2      0
3      0
4      1
      ..
855    1
856    0
857    1
858    0
859    0
Name: class_a_code, Length: 860, dtype: int64

In [203]:
test_labels

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [29]:
# Convert to TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)

2023-07-22 00:10:08.116070: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:

# Initialize the BERT model (make sure the number of labels match your dataset)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(class_a))

# Choose an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-6)

# Choose a loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from tensorflow.keras.callbacks import EarlyStopping


# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', # usually val_loss or val_acc
    mode='min', # the direction that you want to monitor (min for loss, max for accuracy)
    patience=3, # number of epochs to wait before stopping the training
    verbose=1 # verbosity mode
)



In [32]:
# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[early_stopping] # add early stopping to the callbacks
)

# Evaluate the model
results = model.evaluate(test_dataset)

# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset)[0], axis=-1)


Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2023-07-22 00:11:57.712282: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


580/580 [==============================] - 2166s 4s/step - loss: 0.5598 - accuracy: 0.7187 - val_loss: 0.4679 - val_accuracy: 0.7925
Epoch 2/15
580/580 [==============================] - 2147s 4s/step - loss: 0.4359 - accuracy: 0.8027 - val_loss: 0.4396 - val_accuracy: 0.8046
Epoch 3/15
580/580 [==============================] - 2119s 4s/step - loss: 0.3968 - accuracy: 0.8275 - val_loss: 0.4483 - val_accuracy: 0.7946
Epoch 4/15
580/580 [==============================] - 2120s 4s/step - loss: 0.3553 - accuracy: 0.8526 - val_loss: 0.4591 - val_accuracy: 0.7971
Epoch 5/15
580/580 [==============================] - 2113s 4s/step - loss: 0.3232 - accuracy: 0.8679 - val_loss: 0.4972 - val_accuracy: 0.7782
Epoch 00005: early stopping
54/54 [==============================] - 55s 987ms/step - loss: 0.4063 - accuracy: 0.8233


In [33]:
from sklearn.metrics import classification_report


# Decode the label numbers back into original labels
y_test_inverse = encoder.inverse_transform(y_test)
predictions_inverse = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report = classification_report(y_test_inverse, predictions_inverse,
                               target_names=encoder.classes_.astype(str))
print(report)


              precision    recall  f1-score   support

           0       0.90      0.85      0.87       620
           1       0.66      0.75      0.70       240

    accuracy                           0.82       860
   macro avg       0.78      0.80      0.79       860
weighted avg       0.83      0.82      0.83       860



# test dataset with 150 rows

In [27]:
y_test_150

0      0
1      0
2      1
3      0
4      0
      ..
145    0
146    1
147    0
148    0
149    1
Name: class_a_code, Length: 150, dtype: int64

In [28]:
X_test_150

0      'aurora il kanecty please keep looking for and...
1        '     same here i cant hear what she is saying'
2      'geoffreyowens niggas dont really understandle...
3      'trump may be many things but he is entertaini...
4      'kamalaharris may run for president in  but wi...
                             ...                        
145    'conservatives you cannot ride this one outvot...
146    '    beats the hell nj out of me lol dude alre...
147    'watching boomer getting the news that she is ...
148    'brexit is dead not because the other eu state...
149    'christian america  if we go by trumps example...
Name: tweet, Length: 150, dtype: object

In [222]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_150 = encoder.transform(y_test_150)
test_encodings_150 = tokenizer(list(X_test_150), truncation=True, padding=True, max_length=256)


In [223]:
test_dataset_150 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150),
    test_labels_150
)).batch(16)

In [224]:
predictions_150 = tf.argmax(model.predict(test_dataset_150)[0], axis=-1)

In [225]:
y_test_inverse_150 = encoder.inverse_transform(y_test_150)
predictions_inverse_150 = encoder.inverse_transform(predictions_150.numpy())

# Generate the classification report
report_150 = classification_report(y_test_inverse_150, predictions_inverse_150,
                               target_names=encoder.classes_.astype(str))
print(report_150)

              precision    recall  f1-score   support

           0       0.92      0.85      0.89       100
           1       0.74      0.86      0.80        50

    accuracy                           0.85       150
   macro avg       0.83      0.85      0.84       150
weighted avg       0.86      0.85      0.86       150



In [34]:
model.save('my_model_agrresive')

2023-07-22 13:53:52.370386: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_model_agrresive/assets


INFO:tensorflow:Assets written to: my_model_agrresive/assets


In [29]:
test_tweet_a_new=pd.read_csv('../raw_data/testset-levela.tsv', delimiter='\t', encoding='utf-8')
test_label_a_new=pd.read_csv('../raw_data/labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])

test_tweet_a_new = test_tweet_a_new.merge(test_label_a_new, on='id')

test_tweet_a_new = test_tweet_a_new.drop(columns='id')



In [30]:
test_tweet_a_new

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,#Watching #Boomer getting the news that she is...,NOT
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...
855,#DespicableDems lie again about rifles. Dem Di...,OFF
856,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,3 people just unfollowed me for talking about ...,OFF
858,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [31]:
test_tweet_off = test_tweet_a_new[test_tweet_a_new['class_a']=='OFF']

In [32]:
test_tweet_off

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
5,. . . What the fuck did he do this time?,OFF
7,@USER Do you get the feeling he is kissing @US...,OFF
14,@USER Nigga ware da hits at,OFF
...,...,...
842,"#Antifa are mentally unstable cowards, pretend...",OFF
847,@USER @USER And Browning looked like dog shit ...,OFF
853,All two of them taste like ass. URL,OFF
855,#DespicableDems lie again about rifles. Dem Di...,OFF


In [33]:
test_tweet_off['tweet'][0]

'#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL'

In [34]:
test_tweet_off.index

Int64Index([  0,   4,   5,   7,  14,  15,  27,  29,  30,  38,
            ...
            826, 827, 833, 834, 835, 842, 847, 853, 855, 857],
           dtype='int64', length=240)

In [35]:
index_list = list(test_tweet_off.index)
print(index_list)

[0, 4, 5, 7, 14, 15, 27, 29, 30, 38, 39, 40, 43, 47, 48, 51, 59, 61, 63, 64, 70, 72, 77, 80, 81, 85, 87, 88, 98, 99, 100, 107, 108, 110, 115, 129, 130, 135, 136, 138, 140, 142, 143, 147, 152, 153, 154, 156, 160, 164, 168, 173, 175, 177, 180, 182, 185, 186, 189, 196, 201, 202, 203, 207, 212, 215, 216, 221, 222, 228, 229, 233, 241, 243, 247, 249, 258, 259, 260, 262, 264, 265, 274, 275, 277, 279, 281, 282, 297, 307, 308, 309, 310, 312, 313, 314, 315, 323, 324, 326, 331, 333, 334, 335, 339, 347, 351, 352, 357, 362, 369, 370, 371, 372, 376, 377, 382, 389, 400, 403, 406, 408, 413, 414, 420, 422, 426, 428, 429, 432, 433, 437, 439, 443, 445, 447, 455, 460, 462, 468, 470, 475, 478, 480, 483, 488, 491, 494, 495, 502, 503, 505, 507, 509, 511, 512, 515, 526, 531, 533, 549, 554, 555, 559, 566, 567, 569, 572, 579, 584, 587, 591, 592, 594, 601, 609, 613, 614, 615, 617, 629, 632, 634, 637, 638, 639, 640, 649, 657, 663, 665, 667, 676, 679, 681, 690, 691, 698, 701, 702, 703, 711, 730, 732, 736, 737, 740

In [47]:
import random
random_indexes = random.sample(index_list, 100)

In [48]:
print(random_indexes)

[810, 334, 313, 475, 203, 782, 737, 282, 762, 420, 160, 826, 736, 533, 483, 755, 629, 835, 732, 347, 601, 432, 730, 326, 164, 422, 85, 249, 98, 413, 100, 377, 531, 129, 480, 308, 265, 173, 447, 443, 212, 408, 749, 478, 147, 153, 433, 613, 260, 617, 312, 216, 63, 445, 775, 572, 307, 505, 437, 800, 180, 51, 470, 30, 554, 138, 228, 791, 495, 14, 241, 681, 277, 333, 207, 426, 156, 842, 140, 667, 215, 185, 247, 676, 765, 526, 591, 491, 323, 406, 638, 751, 47, 264, 59, 460, 7, 814, 833, 258]


In [38]:
print(random_indexes[:50])

[774, 778, 698, 488, 260, 38, 39, 594, 282, 241, 741, 443, 711, 676, 640, 331, 362, 107, 61, 460, 857, 403, 130, 786, 408, 559, 764, 281, 667, 115, 63, 853, 690, 262, 579, 168, 847, 462, 85, 40, 614, 736, 632, 352, 216, 30, 196, 313, 422, 447]


In [39]:
test_tweet_off_new = test_tweet_off.loc[random_indexes]

In [40]:
test_tweet_off_new

,tweet,class_a
774,@USER #Rosie makes me nauseous!🤮 URL,OFF
778,#BREXIT deal HAS been reached - and will be un...,OFF
698,@USER #Carrey is a #pervert himself!,OFF
488,@USER fuck,OFF
260,#Liberals Are Reaching Peak Desperation To Cal...,OFF
...,...,...
48,*voice in my head while I transfer money to my...,OFF
142,@USER Stick to Comedy ... You weren't that fu...,OFF
782,#crazyredhead in very #angry at the #orange #c...,OFF
182,#ArunJaitleyStepDown He is most shameless #FM ...,OFF


In [54]:
test_tweet_off['tweet'][173]

"#Elitist #Actors #NBC #Liberals of #Dems in #Hollywood at the #Emmy Awards a 'toothless' #NBC self-promotion onslaught peppered with #MeToo cracks but it was planned, staged and okay because they are #Democrats #Socialist #Liberals of the #Left #WALKAWAY ! URL"

In [49]:
test_tweet_off_new['tweet'][156]

'#TheArchers I want to be brilliant at Karate so I can kick the shit out of everyone""'

# ChatGPT 3.5 with rephrasing less aggressive

In [85]:
test_tweet_off_new['tweet'][27] ="Let's consider a scenario where DeLauro is put in a police lineup. Some people might point her out due to certain negative perceptions. This situation raises questions about the voters' choices or the possibility of voter fraud. It's essential to remember that everyone's right to vote should be respected, regardless of their gender or appearance. \u2066@USER @USER"

In [88]:
test_tweet_off_new['tweet'][370] ="Recent events involving #Beeto in #Texas serve as a demonstration of how not to run a successful election campaign. #MAGA \u2066@USER #rkba @USER #DVC UR"

In [90]:
test_tweet_off_new['tweet'][334] ="#Conservative Individuals can learn valuable lessons from the gaming community. URL #Conservatives #TriggerWarningRadio"

In [92]:
test_tweet_off_new['tweet'][702] = "I'd like to share my opinion, even though #NoOneAskMe. Regarding the ongoing beef between #CardiB and @USER, it seems like #Offset is involved to some extent. Cardi might be trusting him, considering their relationship. However, it's important for #offset to handle the situation calmly and thoughtfully."

In [94]:
test_tweet_off_new['tweet'][857]="Three people recently unfollowed me after discussing Merlin. Apologies, everyone! I'm still obsessed with Covinsky, but I'm eagerly waiting for a possible sequel announcement for PS I Still Love You. I've run out of witty and funny tweets about To All the Boys I've Loved Before; it's been quite exhausting."

In [96]:
test_tweet_off_new['tweet'][243] = "#Trump's actions have raised concerns about possible obstruction of justice, as some see them as significant evidence. Critics view him as a controversial figure in this regard."

In [99]:
test_tweet_off_new['tweet'][460] = "#Feinstein, it's important to be honest and truthful in your statements. Let's strive to maintain integrity and avoid spreading misinformation. URL"

In [101]:
test_tweet_off_new['tweet'][827] = "#MAGA Can you please share one positive action that #Trump has taken during his time as #president? I'm open to hearing different perspectives. Let's discuss his accomplishments and record objectively."

In [103]:
test_tweet_off_new['tweet'][326] = "#FamiliarWife JH-Oh my goodness, she's quite unpredictable. WJ-Hold on!...yes, I find assistant manager Cha interesting. BH-You two seem compatible. VO-Any issues between you? Just sit here. JH-I advise against it! WJ-Let's live life on our terms. Let's give it a try. News-There's a collision on the highway causing traffic. JH-Woo-jin~ URL"

In [105]:
test_tweet_off_new['tweet'][30] = "#ChristineBlaseyFord is the individual who accused #Kavanaugh. It seems like this accusation is a pattern that #Liberals tend to follow each time. Let's focus on discussing the #ConfirmJudgeKavanaugh process. URL"


In [107]:
test_tweet_off_new['tweet'][221]="#DeclassifyDocuments It's intriguing to observe different perspectives, like Maria's, during discussions. #Outnumbered"

In [109]:
test_tweet_off_new['tweet'][617] = "#LiberalismIsAMentalDisorder It's essential to recognize differing viewpoints. #Hypocrites #DemocratsHateAmerica #DemocratsAreDangerous It's important to engage in constructive dialogue and find common ground. URL"

In [111]:
test_tweet_off_new['tweet'][173] ="#Elitist #Actors #NBC #Liberals at the #Emmy Awards showcased a self-promotion event with some #MeToo references. It's worth noting that this was a planned and staged occasion, which might align with their #Democratic and #Socialist beliefs. Let's remember the importance of respectful dialogue and understanding different perspectives. #WALKAWAY URL"

In [113]:
test_tweet_off_new['tweet'][85] = "#Liberals can be seen differently by some, but let's remember the principle of INNOCENT UNTIL PROVEN GUILTY. It's important to examine both parties if there are concerns about individuals' conduct. Let's strive to have respectful discussions and avoid harsh language. #KavanaughIsInnocent #WalkAwayFromDemocrats. Let's focus on honesty and avoid extreme accusations. URL"

In [117]:
test_tweet_off_new['tweet'][371] ="And sometimes, people can give off the impression that they have money, but in reality, they may not"

In [119]:
test_tweet_off_new['tweet'][567] ="Ford's attorney has mentioned that she's facing death threats, leading her to relocate her family for safety. However, it's essential to seek evidence before making assumptions about who might be behind these threats. Let's approach this situation with a fair and unbiased perspective"

In [121]:
test_tweet_off_new['tweet'][428]="Anyone who finds M*neta's behavior humorous may have a different sense of humor."

In [123]:
test_tweet_off_new['tweet'][740] = "@USER - Opinions about this POTUS vary widely."

In [125]:
test_tweet_off_new['tweet'][439] = "#CnnIsFakeNews @USER @USER, some people have concerns about the network's ratings and credibility. #cnnisajoke #FakeNews #MAGA URL"

In [127]:
test_tweet_off_new['tweet'][408] = "#Hillary and at least 16 other individuals have strong opinions as #AngryDemocrats. Let's discuss #WorldWarQ, #WWG1WGA, #MAGA, #RedTsunami, #AmericaFirst, #WalkAway, and #Truth. URL"

In [129]:
test_tweet_off_new['tweet'][420] ="@USER @USER As I've mentioned before, some people believe that media and financial influences, like Soros and Waters, are contributing to increased tensions in society."

In [131]:
test_tweet_off_new['tweet'][638] ="#Liberals are often known for their strategic tactics to achieve their goals. 👇👇 URL"

In [133]:
test_tweet_off_new['tweet'][663]="#HonourSacrificeBoycottMatch I have concerns about the impact on human rights. I hope all parties involved find some satisfaction. Let's engage in cricket with them and express gratitude for their efforts to support our soldiers. We need to address the situation thoughtfully; even smaller nations like Pakistan should not be underestimated."

In [135]:
test_tweet_off_new['tweet'][377] = "#Dems have shown a concerning pattern of believing accusations against conservatives while dismissing those against liberals. There is a need for fair and impartial treatment in such cases. URL"

In [137]:
test_tweet_off_new['tweet'][775]="#FemiNazis, who are often associated with Liberals, seem to emphasize #MeToo only when accusing non-Liberal men. The situation may seem perplexing. #Bizarro #KAG2018 ❤️🇺🇸 URL"

In [139]:
test_tweet_off_new['tweet'][502] ="@USER One possible explanation is that he may be tone-deaf to the situation. It's challenging to remain understanding when he and others continue to behave this way."

In [141]:
test_tweet_off_new['tweet'][468] ="#Nigeria #Naija #9ja 'Atiku criticizes Buhari, calling him one of the least competent leaders in Nigeria's history' - Read more on this via URL provided by championnewsng."

In [143]:
test_tweet_off_new['tweet'][177] ="#BlueWave #DumpTrump ...show love and seek to cooperate or find common ground with conservatives/Trump supporters while advocating for the rights of marginalized groups and supporting women's rights to make choices about their own bodies..."

In [145]:
test_tweet_off_new['tweet'][180] ="#NightMayor's DT cycle track can be repurposed for various community activities, like setting up tennis courts for Antifa inside Crescent Heights, until we address concerns related to Nenshi's vape shop."

In [148]:
test_tweet_off_new['tweet'][422] ="#Texas Ted Cruz's campaign has been sending solicitations that some recipients mistake for official county summons. This has caused confusion, particularly among elderly residents. Some people find this approach questionable. URL"

In [150]:
test_tweet_off_new['tweet'][262] = "$uicideboy$ ON HALLOWEEN! Get ready for a seriously spooky time!"

In [152]:
test_tweet_off_new['tweet'][335] ="#Barbara Boxer: When discussing serious issues like rape, it's crucial for everyone, including liberals, to provide solid evidence. Personal accounts alone might not be sufficient. On a related note, without evidence, making accusations could be misleading. Let's focus on respectful discussions. URL"

In [154]:
test_tweet_off_new['tweet'][774] ="@USER #Rosie's actions or statements sometimes make me feel uncomfortable. URL"

In [156]:
test_tweet_off_new['tweet'][555] ="Anyway, this game isn't really my cup of tea :)"

In [158]:
test_tweet_off_new['tweet'][657] ="#Bakersfield is an example of why some people advocate for gun control. My thoughts are with the victims, except for the gunman himself. I hope justice is served"

In [160]:
test_tweet_off_new['tweet'][201] ="@USER These seem to be coordinated attacks from individuals with extreme views. It's unfortunate that such incidents may escalate further before the election. Some people criticize the President for not being a traditional politician, even though they fear potential consequences for the economy and their 401 plans. Let's hope for more reasoned and respectful discussions."

In [162]:
test_tweet_off_new['tweet'][222] ="#PoliticsInFocus Hey Donald! Let's remember we all have different perspectives, no need to talk down to me! URL'"

In [164]:
test_tweet_off_new['tweet'][511] ="#Kavanaugh has been influenced by Anthony Kennedy, known for delivering opinions that upset liberals. Some view him as an attempt by Trump to convince conservatives that the #SCOTUS is leaning right. If @USER were strategic, they would work to support BK's confirmation. However, it seems unlikely. #TuesdayThoughts URL"

In [166]:
test_tweet_off_new['tweet'][549] ="#NEWS Jeff Sessions: In his perspective, following the guidance of the ACLU, Antifa, and Black Lives Matter may lead to more negative outcomes. URL #CNBC"

In [168]:
test_tweet_off_new['tweet'][509] = "...how about death metal? I'm eager to discover more death metal music... Can you please suggest some good places to find it? URL"

In [170]:
test_tweet_off_new['tweet'][587] ="@USER #ThoseThatAreTheLoudest might be concerned, but many believe that justice will prevail. Let's remember to focus on the facts. #Bono and the #ClintonFoundation have raised some questions, and people have expressed skepticism about their actions. #PedoWood #Haiti URL"

In [172]:
test_tweet_off_new['tweet'][771] = "#Conservatives might not be as informed as some may hope, that's the main idea behind my tweet."

In [174]:
test_tweet_off_new['tweet'][437] = "#MSNBC #Hardball @USER Bingo, you've hit the mark. It seems evident that he holds a grudge against President Obama, likely stemming from the White House correspondents dinner. This could be a significant factor in his negative feelings towards the former President."

In [176]:
test_tweet_off_new['tweet'][800] ="#Kavanaugh If #BlaseyFord's credibility is questioned, there's a possibility of another accuser coming forward. Some people criticize liberals for their consistent approach in such situations. #LiberalismIsAMentalDisorder"

In [178]:
test_tweet_off_new['tweet'][108] = "Amazon is now offering to ship live Christmas trees to your doorstep. But the big question is, will they stay and help with putting on the lights? URL"

In [180]:
test_tweet_off_new['tweet'][207] ="#JoinTheFight Join the discussion about concerns regarding certain political ideologies. Some people feel that Democrats and Liberals are promoting ideas that they believe could be harmful to America. Let's engage in constructive conversations. #JoinTheFightAgainstCommunism URL"

In [182]:
test_tweet_off_new['tweet'][609] ="#DeSantis=DeRacist, while some may view him as a questionable choice, he chose to distance himself from the President's comments about deaths in Puerto Rico from Hurricane Maria. @USER @USER #whataputz"

In [184]:
test_tweet_off_new['tweet'][156] ="#TheArchers I want to excel in Karate and improve my skills for self-defense and personal growth."

In [185]:
test_tweet_off_new_01 = test_tweet_off_new.loc[random_indexes[:50]]

In [186]:
test_tweet_off_new_01

,tweet,class_a
27,Let's consider a scenario where DeLauro is put...,OFF
370,Recent events involving #Beeto in #Texas serve...,OFF
334,#Conservative Individuals can learn valuable l...,OFF
702,"I'd like to share my opinion, even though #NoO...",OFF
857,Three people recently unfollowed me after disc...,OFF
243,#Trump's actions have raised concerns about po...,OFF
460,"#Feinstein, it's important to be honest and tr...",OFF
827,#MAGA Can you please share one positive action...,OFF
326,"#FamiliarWife JH-Oh my goodness, she's quite u...",OFF
30,#ChristineBlaseyFord is the individual who acc...,OFF


In [187]:
test_tweet_not = test_tweet_a_new[test_tweet_a_new['class_a']=='NOT']

In [188]:
random_100_rows = test_tweet_not.sample(n=100)

# Build the new DataFrame using the randomly selected rows
test_tweet_not_new = pd.DataFrame(random_100_rows)

In [189]:
test_tweet_not_new

,tweet,class_a
565,#2A #ArT #1A #KAG #MAGA #PATRiOT OUR ALFA WARi...,NOT
441,"#Chefs, are you in need of some new pans? Find...",NOT
770,"..While reading this, I feel like I am hearing...",NOT
728,#Chequers is dead. D.e.a.d. Time to move on. #...,NOT
600,#CNN irrationally argues 4 legalising #abortio...,NOT
...,...,...
20,"#BREAKING: After a week-long trial, a Linn Cou...",NOT
729,#Auspol Flawed though they were in many ways a...,NOT
596,#BBCbias #wato Conservatives built more th...,NOT
304,#Obasrants If you are in a band and you don’...,NOT


In [190]:
merged_df = pd.concat([test_tweet_not_new, test_tweet_off_new_01], ignore_index=True)

# Shuffle the index
test_shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)

In [191]:
test_shuffled_df

,tweet,class_a
0,#Conservatives are going to make hay out of #T...,NOT
1,#Chequers is dead. D.e.a.d. Time to move on. #...,NOT
2,#Texas Ted Cruz's campaign has been sending so...,OFF
3,#NBA2KFreeAgent We are looking for a 3rd membe...,NOT
4,"#NEWS Jeff Sessions: In his perspective, follo...",OFF
...,...,...
145,#Season2 #scary Er while we are on the subject...,NOT
146,#HIAC: Why is it this match too with Becky vs ...,NOT
147,#SupernaturalTNT #TheManWhoWouldBeKing OMG! @...,NOT
148,#BoycottLeviStrauss Levi Strauss took a public...,NOT


In [192]:
clean_tweets(test_shuffled_df)

/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_93436/3189604534.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_93436/3189604534.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [193]:

test_shuffled_df.update(test_shuffled_df[['tweet']].applymap('\'{}\''.format))

test_shuffled_df.head()

,tweet,class_a
0,'conservatives are going to make hay out of tr...,NOT
1,'chequers is dead dead time to move on superca...,NOT
2,'texas ted cruzs campaign has been sending sol...,OFF
3,'nbakfreeagent we are looking for a rd member ...,NOT
4,'news jeff sessions in his perspective followi...,OFF


In [194]:
test_shuffled_df["class_a_code"] = encoder.fit_transform(test_shuffled_df["class_a"])

In [195]:
X_test_shuffled = test_shuffled_df['tweet']
y_test_shuffled = test_shuffled_df['class_a_code']

In [197]:
X_test_shuffled

0      'conservatives are going to make hay out of tr...
1      'chequers is dead dead time to move on superca...
2      'texas ted cruzs campaign has been sending sol...
3      'nbakfreeagent we are looking for a rd member ...
4      'news jeff sessions in his perspective followi...
                             ...                        
145    'season scary er while we are on the subject d...
146    'hiac why is it this match too with becky vs c...
147    'supernaturaltnt themanwhowouldbeking omg   yo...
148    'boycottlevistrauss levi strauss took a public...
149    'osaka says the crowd was noisy and i really d...
Name: tweet, Length: 150, dtype: object

In [202]:
y_test_shuffled

0      0
1      0
2      1
3      0
4      1
      ..
145    0
146    0
147    0
148    0
149    0
Name: class_a_code, Length: 150, dtype: int64

In [207]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_shuffled = encoder.transform(y_test_shuffled)

In [208]:
test_encodings_shuffled = tokenizer(list(X_test_shuffled), truncation=True, padding=True, max_length=256)

In [211]:
test_dataset_shuffled = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_shuffled),
    test_labels_shuffled
)).batch(16)

In [212]:
# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset_shuffled)[0], axis=-1)


In [213]:
# Decode the label numbers back into original labels
y_test_inverse_shuffled = encoder.inverse_transform(y_test_shuffled)
predictions_inverse_shuffled = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report_gpt3 = classification_report(y_test_inverse_shuffled, predictions_inverse_shuffled,
                               target_names=encoder.classes_.astype(str))
print(report_gpt3)

              precision    recall  f1-score   support

           0       0.64      0.84      0.73       100
           1       0.16      0.06      0.09        50

    accuracy                           0.58       150
   macro avg       0.40      0.45      0.41       150
weighted avg       0.48      0.58      0.51       150



# ChatGPT 3 with rephrasing


In [229]:
print(random_indexes[50:])

[308, 216, 732, 376, 382, 339, 47, 99, 797, 282, 228, 362, 701, 480, 115, 136, 77, 51, 681, 749, 389, 260, 258, 100, 233, 554, 810, 483, 614, 778, 107, 613, 175, 307, 634, 313, 182, 154, 855, 352, 853, 323, 40, 38, 531, 135, 281, 737, 823, 413]


In [324]:
test_tweet_off_new['tweet'][413] 

'#Antifa is recruiting for a designated #terrorist organization URL'

In [231]:
test_tweet_off_new['tweet'][308] = "Alright, let's consider the situation. It's true that some players have scored 15+ goals in Eredivisie, but some might argue that it's not the most challenging league. On the other hand, Bundesliga teams have proven to be competitive in European competitions. As for Mexican talent, it's been some time since someone like Pulisic emerged. Nevertheless, there's always hope for the future, and who knows, Mexico might produce exceptional talents in due time."

In [233]:
test_tweet_off_new['tweet'][216] ="It seems like both males and females are dealing with similar situations, and that's something to consider. The URL might be a shared experience for both genders."

In [235]:
test_tweet_off_new['tweet'][732] = "Addressing the topic of #LiberalHypocrisy and #TacoBell, some individuals question why others are against illegal immigration. They mention an incident involving a Taco Bell employee responding with 'No Habla Ingles!' when asked."

In [237]:
test_tweet_off_new['tweet'][376]  ="It's been an incredible seven years! #MyTwitterAnniversary URL"

In [239]:
test_tweet_off_new['tweet'][382] ="The situation surrounding #ChristineBlaseyFord has drawn attention, and some suggest that she is facing harassment from certain groups. They point to the involvement of Antifa, seen as the radical arm of the Democratic Party, allegedly backed by George Soros. The hashtag #VoteRedToSaveAmerica is being used in this context."

In [241]:
test_tweet_off_new['tweet'][339] ="On #ConstitutionDay, there's an interesting contrast between some conservative members of the alt-right who criticize efforts for #GunControlNow, accusing it of damaging the Constitution, while expressing concerns about non-White foreigners seeking to live in this country. The situation sparks debate on constitutional interpretations and immigration policies."

In [243]:
test_tweet_off_new['tweet'][47] ="#Conservatives, it's disheartening to witness such disrespectful interactions like the one with @USER. URL"

In [245]:
test_tweet_off_new['tweet'][99]  ="Some people find #SerenaWilliams to be overly confident, making it challenging to enjoy both watching her play and listening to her speak."

In [247]:
test_tweet_off_new['tweet'][797] ="If you want to encourage people to leave your home, perhaps you could try using humor or lighthearted jokes, like pretending to be a vampire, without causing harm or discomfort. It's essential to find a friendly and non-threatening way to address the situation."

In [249]:
test_tweet_off_new['tweet'][282] = "The ongoing #CrowderConfronts series continues to demonstrate that when conservatives respond with truth and facts, the so-called SJW bullies appear less intimidating and substantial. It encourages others not to be afraid and emphasizes the importance of confronting issues with honesty and evidence. URL"

In [251]:
test_tweet_off_new['tweet'][228] ="'#CarbonTax proves effective and promises to significantly increase money in the pockets of Canadians. .@USER .@USER .@USER .@USER Back in 2015, part of Trudeau's successful campaign was centered around a national carbon tax, which played a role in the Conservative party's defeat. They face a similar challenge in 2019, and some are determined to ensure their loss once again. URL'"

In [254]:
test_tweet_off_new['tweet'][362] ="'#Repost theleadersmindset • • • 📍Hello Leaders!🔥 ✔️Type 'READY' if you are prepared to embrace these steps. - 'Set goals, keep them to yourself, crush them with determination, celebrate your successes… URL'"

In [256]:
test_tweet_off_new['tweet'][701] ="'@USER nigga are you stupid your trash dont play with him play with your bitch 😂'"

In [258]:
test_tweet_off_new['tweet'][480] ="#FailedDemFeinstein's continued presence in politics is questionable, and some believe she should have stepped down years ago. Her recent actions, like blaming the GOP in the Kavanaugh accuser situation while admitting uncertainty about the truthfulness, add to her perceived disgrace. URL"

In [260]:
test_tweet_off_new['tweet'][115] ="#Conservatives Govt ' @USER caused immense hardship': whistleblowers report being threatened by a regulator; Weren't they supposed to safeguard whistleblowers? URL"

In [262]:
test_tweet_off_new['tweet'][136] ="@USER #WalkAwayFromAllDemocrats Michelle had her 8 years, and many believe we don't need more of her advice. We now have a new Sheriff in Town who's #MAGA. It's crucial for everyone, regardless of color or gender (AKA all Americans), to get out and VOTE! URL"

In [264]:
test_tweet_off_new['tweet'][77] ="#ANTIFA is seen as allied with The Democratic Socialists of America (DSA), whom #JamesOKeefe has revealed as having infiltrated various government positions with the aim of undermining #America. #Trump #MAGA🇺🇸 #DeepState URL"

In [266]:
test_tweet_off_new['tweet'][51] ="'#Kids Beating #Paraplegic #Trump #Boy!  #Left Teach Their #Children Hate Violence!  #schools #teachers #Democrats #Liberals #WalkAway #Moms #Mother #NEA #teachthemyoung #Christian #Catholic #Jews #Evangelicals #family #BlackWomen #BlackTwitter #Blacks #News #Media #Disabled #MAGA URL'"

In [268]:
test_tweet_off_new['tweet'][681] ="Can you believe it? A hundred rounds! Goon, you must be kidding me! 🤣"

In [270]:
test_tweet_off_new['tweet'][749] ="#Antifa plans to counter the fascist NVU in #Amsterdam on September 22. URL"

In [272]:
test_tweet_off_new['tweet'][389] ="50 Cent addresses Joe Budden's actions on Instagram, calling them out as 'bullshit.' URL URL"

In [274]:
test_tweet_off_new['tweet'][260] ="#Liberals are resorting to extreme measures, urging #PhillipRuddock to communicate with #Turnbull and persuade him to assist with #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPMemes URL"

In [276]:
test_tweet_off_new['tweet'][258] ="'#CincinnatiEnquirer finally addresses #GunControl following a tragic #MassShooting right on their doorstep. It took this event for their editors to wake up, but better late than never. URL'"

In [278]:
test_tweet_off_new['tweet'][100] ="Regarding #ColinKaepernick, some people believe he has been unproductive and merely voicing complaints. They think it's time for both black individuals and passionate white liberals, including @USER, to become more aware or stop making noise about the cause. They see Colin as not fitting the role of a Civil Rights hero or martyr by any means."

In [280]:
test_tweet_off_new['tweet'][233] ="During #TheView, Stormy Daniels found herself in a difficult situation with a dollar bill in her face, highlighting the challenges faced by adult film stars. The post also seems to allude to her involvement with the Democratic Party."

In [282]:
test_tweet_off_new['tweet'][554] = "#DavidMiliband relocated to #NewYork to align with #US #Liberals #Democrats, seeking a departure from the hostility towards Jews by #Corbynists."

In [284]:
test_tweet_off_new['tweet'][810] ="'#BCOG @USER portrays a cool, confident, and assertive character who is highly skilled and successful! @USER @USER @USER URL'"

In [286]:
test_tweet_off_new['tweet'][483] ="'@USER @USER @USER Your meme suggests that both Fascism and Antifa were opposed to liberals.'"

In [288]:
test_tweet_off_new['tweet'][614] ="@USER @USER I plan to use that argument in future discussions on gun control, free speech, or taxes. It's essential to promote responsibility, and that's what I advocate for. There's nothing right about justifying murder by denying the life of another or claiming it as a right."

In [290]:
test_tweet_off_new['tweet'][778] ="It appears that a #BREXIT deal has been reached and will be presented at a special summit in November. The question raised is whether @USER has compromised the #UK's position with the EU. People are concerned, hoping that there hasn't been a sellout, as it could have significant consequences for @USER. URL"

In [292]:
test_tweet_off_new['tweet'][107] ="Jeff Sessions suggests that listening to groups like the ACLU, Antifa, and Black Lives Matter could result in an increase in Chicago shootings. URL"

In [294]:
test_tweet_off_new['tweet'][613] ="Lost my acca on the first fight, 0-1. So frustrating."

In [297]:
test_tweet_off_new['tweet'][307] ="#1 son, hitting it out of the park! URL"

In [300]:
test_tweet_off_new['tweet'][313]  = "@USER @USER @USER Many believe she is the most disingenuous person in the Senate. If she were my Senator, I would feel ashamed due to her despicable behavior in the SCOTUS hearings."

In [302]:
test_tweet_off_new['tweet'][182] = "#ArunJaitleyStepDown is trending, with many expressing strong criticism towards him. They consider him the most shameless Finance Minister in the history of India, and his audacity to lie in public is viewed as a disgrace to his position."

In [304]:
test_tweet_off_new['tweet'][154] ="#GOP still appearing hesitant, and some view them as lacking courage. It seems they haven't realized that Liberals will oppose them no matter what they say. Some believe Republicans are either struggling to make strategic decisions or have shifted too far left. #ConfirmKavanaughNow"

In [306]:
test_tweet_off_new['tweet'][855] = "'#DespicableDems are accused of spreading misinformation about rifles. Some claim that Democrats manipulated the law to advocate for gun control during the Kavanaugh confirmation. URL via @USER'"

In [308]:
test_tweet_off_new['tweet'][352] = "'#OrrinHatch's involvement in another woman's charges against a Supreme Court nominee has sparked strong criticism. Some view him as sexist, clueless, and outdated, drawing comparisons to his past behavior during the Anita Hill case. They believe it's time for him to step aside. Good riddance to this Neanderthal!'"

In [310]:
test_tweet_off_new['tweet'][853] ="Both of them have an unpleasant taste. URL"

In [312]:
test_tweet_off_new['tweet'][323] = "#AVALANCHE, I apologize! I acknowledge your presence, and I care about you! I regret my previous behavior and acknowledge that we all make mistakes. #maga"

In [315]:
test_tweet_off_new['tweet'][38] = "#ArianaAsesina? Seriously?! Please, let's not jump to conclusions and spread hate. She's already going through a tough time, and pointing fingers won't help. It was an overdose, and blaming others won't change the tragic outcome. Let's stop seeking someone to blame, and instead, show empathy and support."



In [317]:
test_tweet_off_new['tweet'][531] = "@USER, it's widely known that a genuine leader doesn't stumble or fall like a sack of potatoes into a van..."

In [319]:
test_tweet_off_new['tweet'][135] = "'#BoycottNFL, I have decided to no longer support you. You have lost my loyalty. @USER @USER URL'"

In [321]:
test_tweet_off_new['tweet'][281]  = "'#Conservatives Govt has accumulated debt despite implementing austerity cuts, and during this time, the wealth of the rich has doubled. This highlights the issue of #inequality. URL via @USER'"

In [325]:
test_tweet_off_new['tweet'][413] = "'#Antifa is actively seeking new members for what some perceive as a designated #terrorist organization. URL'"

In [326]:
test_tweet_off_new_02 = test_tweet_off_new.loc[random_indexes[50:]]

In [327]:
merged_df_02 = pd.concat([test_tweet_not_new, test_tweet_off_new_02], ignore_index=True)

# Shuffle the index
test_02_df = merged_df_02.sample(frac=1,random_state=42).reset_index(drop=True)

In [328]:
test_02_df

,tweet,class_a
0,#BaldwinFundraises at K Street Firm Working Pr...,NOT
1,@USER @USER ...than why did you show us how ho...,NOT
2,"Can you believe it? A hundred rounds! Goon, yo...",OFF
3,#osho #quotes via @USER Listen to your being. ...,NOT
4,#BrexitShambles #ThursdayThoughts #eu #NEWS Le...,NOT
...,...,...
145,#Brexit #UKLabour #Labour #Conservatives #Plai...,NOT
146,"#Conservatives, it's disheartening to witness ...",OFF
147,@USER #Google as a fellow Goggler I must say e...,NOT
148,"10 years ago today, Taylor Swift released her ...",NOT


In [329]:
clean_tweets(test_02_df)

/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_93436/3189604534.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
/var/folders/4y/7vss76ld1rj1ybqx7xpskc5m0000gn/T/ipykernel_93436/3189604534.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [330]:
test_02_df.update(test_02_df[['tweet']].applymap('\'{}\''.format))

test_02_df.head()

,tweet,class_a
0,'baldwinfundraises at k street firm working pr...,NOT
1,' than why did you show us how hot she is ',NOT
2,'can you believe it a hundred rounds goon you ...,OFF
3,'osho quotes via listen to your being it is c...,NOT
4,'brexitshambles thursdaythoughts eu news lets ...,NOT


In [331]:
test_02_df["class_a_code"] = encoder.fit_transform(test_02_df["class_a"])

In [332]:
X_test_02 = test_02_df['tweet']
y_test_02 = test_02_df['class_a_code']

In [333]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_02 = encoder.transform(y_test_02)

In [334]:
test_encodings_02 = tokenizer(list(X_test_02), truncation=True, padding=True, max_length=256)

In [335]:
test_dataset_02 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_02),
    test_labels_02
)).batch(16)

In [336]:
# Get the model predictions
predictions_02 = tf.argmax(model.predict(test_dataset_02)[0], axis=-1)

In [337]:
# Decode the label numbers back into original labels
y_test_inverse_02 = encoder.inverse_transform(y_test_02)
predictions_inverse_02 = encoder.inverse_transform(predictions_02.numpy())

# Generate the classification report
report_02 = classification_report(y_test_inverse_02, predictions_inverse_02,
                               target_names=encoder.classes_.astype(str))
print(report_02)

              precision    recall  f1-score   support

           0       0.71      0.84      0.77       100
           1       0.50      0.32      0.39        50

    accuracy                           0.67       150
   macro avg       0.61      0.58      0.58       150
weighted avg       0.64      0.67      0.64       150

